In [5]:
import pandas as pd
import joblib

df = pd.read_csv("tumors.csv")

from sklearn.linear_model import LinearRegression

model = LinearRegression()

X = df[["size", "p53_concentration"]]
y = df["is_cancerous"]

model.fit(X, y)

joblib.dump(model, "model.joblib")

['model.joblib']

In [104]:
import os

to_predict = [-0.0041649365241367, 0.0017850734344602]
#to_predict = [2.7, 3.2]

def produce_linear_regression_c_code():

    model = joblib.load('model.joblib')

    len_thetas = len(model.coef_) + 1

    thetas = f"{model.intercept_}f,"

    for coef in model.coef_:
        thetas += str(coef) + "f,"

    thetas = thetas.strip(",")

    prediction_code = f"float thetas[{len_thetas}] = {{{thetas}}};"

    to_predict_c = ""

    for value in to_predict:
        to_predict_c += str(value) + "f,"

    to_predict_c = to_predict_c.strip(",")

    code = f"""
    #include <stdio.h>

    {prediction_code}
    float prediction(float *features, int n_feature)
    {{
        float res = thetas[0];

        for (int i = 0; i < n_feature; ++i)
            res += features[i] * thetas[i+1];

        return res;
    }}
    int main()
    {{
        float to_predict[2] = {{{to_predict_c}}};

        printf("%f\\n", prediction(to_predict, 2));

        return 0;
    }}
    """

    with open("fichier.c", "w") as f:
        f.write(code)

    if os.system("gcc fichier.c -O3 -o main"):
        print("Compile error")
produce_linear_regression_c_code()

In [105]:
!gcc fichier.c -O3 -o main

In [106]:
produce_linear_regression_c_code()

print("C code output:")
!./main
print("Model output:", model.predict([to_predict]))

C code output:
0.629072
Model output: [0.6290722]


In [25]:
model.predict([list(df.iloc[0, :2])])

array([0.6290722])

In [26]:
list(df.iloc[0, :2])

[-0.0041649365241367, 0.0017850734344602]

In [50]:
model.predict([[-0.0041649365241367, 0.0017850734344602]])

array([0.6290722])